In [1]:
import numpy as np
import codecs, string

with codecs.open("common-english-words.txt", "r", "utf-8") as f:
    common_words = f.read().split(",")


In [2]:
# Create and clean the text
text = "Intelligent behavior in people is a product of the mind. But the mind itself is more like what the human brain does."
def clean(text):
    # Remove stop words
    text = " ".join(list(filter(lambda x: x not in common_words, text.split(" "))))
    # Remove punctutation
    text = "".join(list(filter(lambda x: x not in string.punctuation+"\n\r\t\0", text)))
    # To lower case
    text = text.lower()
    return text
text = clean(text)
text

'intelligent behavior people product mind but mind itself more human brain does'

In [3]:
# Create inverted index
vocab = dict([(key, (1, text.count(key))) for key in set(text.split(" "))])
vocab

{'more': (1, 1),
 'brain': (1, 1),
 'but': (1, 1),
 'intelligent': (1, 1),
 'does': (1, 1),
 'behavior': (1, 1),
 'human': (1, 1),
 'mind': (1, 2),
 'product': (1, 1),
 'itself': (1, 1),
 'people': (1, 1)}

In [4]:
# Construct a (normal) inverted index
# For one document this is just a frequency list
def gen_idx(corpus):
    # Initiate the index as a dict('term', dict('doc', num_occ))
    idx_list = dict([(key, {}) for key in set(" ".join(corpus).split(" "))])
    for doc_idx, doc in enumerate(corpus):
        # Increment number of occurrences for each occurrence
        for term in doc.split(" "):
            if doc_idx not in idx_list[term].keys():
                idx_list[term][doc_idx] = 0
            idx_list[term][doc_idx] += 1
    return idx_list
gen_idx([text])

{'more': {0: 1},
 'brain': {0: 1},
 'but': {0: 1},
 'intelligent': {0: 1},
 'does': {0: 1},
 'behavior': {0: 1},
 'human': {0: 1},
 'mind': {0: 2},
 'product': {0: 1},
 'itself': {0: 1},
 'people': {0: 1}}

In [5]:
def gen_idx_block(corpus, block_size=3):
    # Initiate the index as a dict('term', dict('doc', [block_ids]))
    idx_list = dict([(key, {}) for key in set(" ".join(corpus).split(" "))])
    corpus_blocks = []
    for doc_idx, doc in enumerate(corpus):
        # Generate blocks
        blocks = [doc.split(" ")[block_size*i:block_size*i+block_size] for i in range(len(doc.split(" "))//block_size+1)]
        blocks = list(filter(lambda x: len(x)>0, blocks))
        corpus_blocks.append(blocks)
        # For each distinct term in the document
        for term in set(doc.split(" ")):
            if doc_idx not in idx_list[term].keys():
                idx_list[term][doc_idx] = []
            # Find occurrences and add block to block list:
            for block_idx, block in enumerate(blocks):
                if term in block:
                    idx_list[term][doc_idx].append(block_idx)

    return idx_list, corpus_blocks

In [6]:
gen_idx_block([text])[0]

{'more': {0: [2]},
 'brain': {0: [3]},
 'but': {0: [1]},
 'intelligent': {0: [0]},
 'does': {0: [3]},
 'behavior': {0: [0]},
 'human': {0: [3]},
 'mind': {0: [1, 2]},
 'product': {0: [1]},
 'itself': {0: [2]},
 'people': {0: [0]}}

In [7]:
# 'word': {doc_id: [block_indices]}
# Indexing using block addressing

print(text)
idx, blocks = gen_idx_block([text], block_size=3)
for bidx, block in enumerate(blocks[0]):
    print(bidx,block)
idx

intelligent behavior people product mind but mind itself more human brain does
0 ['intelligent', 'behavior', 'people']
1 ['product', 'mind', 'but']
2 ['mind', 'itself', 'more']
3 ['human', 'brain', 'does']


{'more': {0: [2]},
 'brain': {0: [3]},
 'but': {0: [1]},
 'intelligent': {0: [0]},
 'does': {0: [3]},
 'behavior': {0: [0]},
 'human': {0: [3]},
 'mind': {0: [1, 2]},
 'product': {0: [1]},
 'itself': {0: [2]},
 'people': {0: [0]}}

## Creating a Suffix Tree

In [8]:
# Vocabulary suffix trie
from typing import Any


class Node:
    def __init__(self, data=None, index=-1):
        self.children = {}
        self.data = data
        self.index = index

txt = "missing mississippi"
suffixes = [txt[i:] for i in range(len(txt))]
suffixes

class SuffixTrie:

    def __call__(self, *args: Any, **kwds: Any) -> Any:
        print("test")

    def create_tree(self, txt):
        self.root = self.build_tree(txt)
        self.root = self.reduce(self.root, txt)

    def build_tree(self, txt):
        txt += "$"
        root = Node()
        current = root
        for i in range(len(txt)):
            current = root
            for j in range(i, len(txt)):
                c = txt[j]
                if c not in current.children:
                    newNode = Node(index=j-(j-i)+1, data=c if c != " " else "_")
                    current.children[c] = newNode
                current = current.children[c]
        return root

    def search(self, txt, count=0):
        current = root
        if len(txt) <= 0:
            if txt not in current.children:
                return False
            return current.children[txt].index
        for c in txt:
            if c in current.children:
                current = current.children[c]
            else:
                return self.search(txt[current.index:len(txt)+1], count+1)
        return current.index-1

    def reduce(self, current: Node, txt):
        idx = self.search()

        for child in current.children:
            self.reduce(child)

root = SuffixTrie()


In [9]:
txt = "missing mississippi"

def search(key, count=0):
    print(key)
    current = root
    if len(key) <= 0:
        if key not in current.children:
            return False
        print(current.children)
        return current.children[key].index
    for c in key:
        if c in current.children:
            current = current.children[c]
        else:
            return search(txt[current.index:len(key)+1], count+1)
    return current.index-1

# search("issing")

## Listing on corpus

In [10]:
corpus = [
    "Although we know much more about the human brain than we did even",
    "ten years ago, the thinking it engages in remains pretty much a total",
    "mystery. It is like a big jigsaw puzzle where we can see many of the",
    "pieces, but cannot yet put them together. There is so much about us",
    "that we do not understand at all.",
]
corpus = [clean(text) for text in corpus]
corpus

['although know much more human brain even',
 'ten years ago thinking engages remains pretty much total',
 'mystery it big jigsaw puzzle see many',
 'pieces put together there much',
 'understand all']

In [11]:
# index, blocks = gen_idx_block(corpus, block_size=3)
index = gen_idx(corpus)
index

{'understand': {4: 1},
 'pieces': {3: 1},
 'see': {2: 1},
 'engages': {1: 1},
 'mystery': {2: 1},
 'it': {2: 1},
 'jigsaw': {2: 1},
 'much': {0: 1, 1: 1, 3: 1},
 'there': {3: 1},
 'more': {0: 1},
 'although': {0: 1},
 'years': {1: 1},
 'put': {3: 1},
 'know': {0: 1},
 'together': {3: 1},
 'many': {2: 1},
 'total': {1: 1},
 'brain': {0: 1},
 'ten': {1: 1},
 'all': {4: 1},
 'puzzle': {2: 1},
 'ago': {1: 1},
 'even': {0: 1},
 'big': {2: 1},
 'remains': {1: 1},
 'human': {0: 1},
 'thinking': {1: 1},
 'pretty': {1: 1}}

In [12]:
index1 = index['much']
index1

{0: 1, 1: 1, 3: 1}

In [13]:
tf = np.array(list(index1.values()))
df = len(index1)
idf = np.log2(len(corpus) / df)
tf*idf

array([0.73696559, 0.73696559, 0.73696559])

# Test

In [14]:
from nltk.stem.porter import PorterStemmer
import string

# This is used for preprocessing of both the corpus and queries
def preprocessing(text):
    # Initiate stemmer
    stemmer = PorterStemmer()

    # Define unwanted characters (punctuation)
    bad_chars = string.punctuation+"\n\r\t"

    # Clean, tokenize and stem text
    new_text = text = text.lower() # all lower case
    new_text = "".join(list(filter(lambda x: x not in bad_chars, new_text))) # remove unwanted chars
    new_text = new_text.split(" ") # tokenize (split into words)
    new_text = list(filter(lambda c: len(c) > 0, new_text)) # remove empty strings
    new_text = [stemmer.stem(word) for word in new_text] # perform stemming
    new_text = " ".join(new_text)
    return new_text

In [15]:
corpus = []
for i in range(1,7):
    with open(f"./DataAssignment4/Text{i}.txt") as f:
        corpus.append(f.read())
corpus = [preprocessing(doc) for doc in corpus]
index = gen_idx(corpus)
index

{'differ': {1: 1, 4: 7},
 'sentenc': {3: 3},
 'must': {1: 2, 5: 3},
 'indescrib': {0: 3},
 'sens': {0: 3},
 'size': {1: 1},
 'irregular': {1: 1},
 'itch': {1: 2},
 'certainli': {1: 2},
 'river': {3: 4},
 'last': {3: 3},
 'he': {1: 40, 5: 4},
 'fall': {1: 1},
 'true': {1: 1},
 'writer': {3: 3},
 'member': {4: 7},
 'seiz': {2: 3},
 'system': {5: 3},
 'food': {1: 1},
 'sink': {0: 3},
 'effort': {1: 1},
 'initi': {3: 3},
 'think': {0: 3, 1: 2},
 'exquisit': {0: 3, 2: 3},
 'possess': {0: 4},
 'travel': {1: 4},
 'sort': {1: 1},
 'float': {0: 3},
 'vixen': {2: 6},
 'dream': {1: 2},
 'can': {1: 2, 5: 3},
 'far': {3: 14},
 'came': {3: 3},
 'a': {0: 18, 1: 23, 2: 25, 3: 29, 4: 21, 5: 18},
 'window': {1: 1},
 'move': {1: 1, 2: 3},
 'slid': {1: 1},
 'readi': {1: 1},
 'frequent': {5: 3},
 'plant': {0: 3},
 'quiz': {2: 33},
 'howev': {1: 1, 3: 3},
 'mistress': {0: 3},
 'kvetch': {2: 3},
 'amaz': {2: 3},
 'almost': {3: 3},
 'jock': {2: 3},
 'get': {1: 6, 2: 3},
 'jay': {2: 3},
 'achiev': {4: 7},
 'ha

In [16]:
sorted([(k, list(v.keys())) for k, v in index.items()], key=lambda x: len(x[1]))[-10:]

[('that', [0, 1, 3, 4, 5]),
 ('wa', [0, 1, 2, 3, 5]),
 ('be', [0, 1, 3, 4, 5]),
 ('a', [0, 1, 2, 3, 4, 5]),
 ('of', [0, 1, 2, 3, 4, 5]),
 ('the', [0, 1, 2, 3, 4, 5]),
 ('to', [0, 1, 2, 3, 4, 5]),
 ('and', [0, 1, 2, 3, 4, 5]),
 ('is', [0, 1, 2, 3, 4, 5]),
 ('in', [0, 1, 2, 3, 4, 5])]

In [17]:
def retrieve_raw_indexes(terms):
    indexes = []
    for term in terms:
        if term in index:
            indexes.append(index[term])
        else:
            indexes.append({})
    return indexes

In [18]:
import re
def parse_query(query):
    # Tokenize the query
    query = query.replace(" -", " NOT ")
    tokens = re.findall(r'(?:AND|OR|NOT|\(|\)|[a-zA-Z0-9]+)', query)
    return parse_expression(tokens)

def parse_expression(tokens):
    current_expression = []

    while tokens:
        token = tokens.pop(0)

        if token == "(":
            # Start a new nested expression
            nested_expression = parse_expression(tokens)
            current_expression.append(nested_expression)

        elif token == ")":
            # End the current expression
            break
        else:
            current_expression.append(token)

    return current_expression

# Example usage:
query = "brown AND (fox bear) -(bubble cat)"
parsed_query = parse_query(query)
parsed_query

['brown', 'AND', ['fox', 'bear'], 'NOT', ['bubble', 'cat']]

In [19]:
def get_docs_by_intersection(terms):
    # Get the indexes of the AND terms
    indexes = retrieve_raw_indexes(terms)
    indexes = sorted(indexes, key=len) # sort for efficiency
    if not len(indexes):
        return []
    # Accumulate docs if they are in all other terms
    docs = []
    for key in indexes[0].keys(): # for all docs in first index
        if all([key in idx.keys() for idx in indexes]): # the doc is in all other indexes
            docs.append(key)
    return docs
id1 = [{2: 1, 3: 7}]
query = ["brown", "fox"]
get_docs_by_intersection(["brown", "fox", "bear"])

[]

In [20]:
def get_docs_by_union(terms):
    # Get the indexes of the NOT terms
    indexes = retrieve_raw_indexes(terms)
    if not len(indexes):
        return []
    # Accumulate docs of the NOT terms
    docs = set()
    for index in indexes:
        docs.update(index.keys())
    return list(docs)

get_docs_by_union(["brown", "fox", "bear"])

[0, 1, 2]

In [21]:
def retrieve_docs(term):
    term = preprocessing(term)
    if term in index:
        return set(index[term].keys())
    return set()

In [28]:
def search_recursive(query, doc_set:set=set()):
    current_context = set()
    operator = None

    for term in query:
        if isinstance(term, list):
            # Recursively process sub-expression
            sub_result = search_recursive(term, doc_set)
            if operator == 'AND':
                doc_set.intersection_update(sub_result)
            elif operator == 'NOT':
                doc_set.difference_update(sub_result)
            else: # default OR
                doc_set.update(sub_result)
        elif term in {'AND', 'OR', 'NOT', '-'}:
            # Set the current operator
            operator = term
        else:
            # Process term
            term_docs = retrieve_docs(term)  # Replace with your actual retrieval function
            if operator == 'AND':
                if not current_context:
                    current_context = term_docs
                else:
                    current_context.intersection_update(term_docs)
            elif operator == 'OR':
                current_context.update(term_docs)
            elif operator == 'NOT':
                current_context.difference_update(term_docs)
            else:
                current_context.update(term_docs)
    
    # After processing all terms and operators in the query, update the document set
    doc_set.update(current_context)
    
    return doc_set


def search(query):
    parsed_query = parse_query(query)
    return list(search_recursive(parsed_query))

# query = "brown AND (fox) -(bubble cat)"
# query = "enjoy AND bears"
# docs = search(query)
# docs

In [43]:
query = "(claim AND banan)"
docs = search(query)

docs_full = [corpus[i] for i in docs]
docs_full = [preprocessing(doc) for doc in docs_full]
vocab = set(" ".join(docs_full).split(" "))

query = preprocessing(query).split(" ")
query = list(filter(lambda x: x in vocab, query))
len(vocab), docs, query

(516, [1, 5], ['claim', 'and'])

In [44]:
for doc in docs:
    print([index[term][doc] if doc in index[term] else 0 for term in query])


[1, 26]
[3, 42]


In [45]:
tfs = np.array([[(index[term][doc] if doc in index[term] else 0) for term in query] for doc in docs])
dfs = np.array([len(index[term]) for term in query])
idfs = np.log2(len(corpus) / dfs)
tf_idf = np.sum(tfs*idfs, axis=1)
scores_docs = {doc: score for doc, score in zip(docs, tf_idf)}
sorted_docs = dict(sorted(scores_docs.items(), key=lambda x: x[1])[::-1])
for k, v in sorted_docs.items():
    print(f"ID: {k}, Score: {v:.4f}, Abstract: {corpus[k]}")


ID: 5, Score: 4.7549, Abstract: but i must explain to you how all thi mistaken idea of denounc pleasur and prais pain wa born and i will give you a complet account of the system and expound the actual teach of the great explor of the truth the masterbuild of human happi no one reject dislik or avoid pleasur itself becaus it is pleasur but becaus those who do not know how to pursu pleasur ration encount consequ that are extrem pain nor again is there anyon who love or pursu or desir to obtain pain of itself becaus it is pain but becaus occasion circumst occur in which toil and pain can procur him some great pleasur to take a trivial exampl which of us ever undertak labori physic exercis except to obtain some advantag from it but who ha ani right to find fault with a man who choos to enjoy a pleasur that ha no annoy consequ or one who avoid a pain that produc no result pleasur on the other hand we denounc with righteou indign and dislik men who are so beguil and demor by the charm of ple

In [26]:
tfs = np.array([[index[term][doc] for term in vocab] for doc in docs])
dfs = np.array([len(index[term]) for term in vocab])
idfs = np.log2(len(corpus) / dfs)

tf_idf = tfs*idfs

scores_docs = zip(tf_idf, docs)

sorted_docs = sorted(scores_docs, key=lambda x: x[0])[::-1]

KeyError: 0